In [1]:
import os
import pytorch_lightning as pl
import torch
from model.model_interface import LLM
import torch.utils.data as tud
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import torch.nn.functional as F
from tqdm.notebook import tqdm

torch.set_float32_matmul_precision('medium')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

LLM Config

In [2]:
llm_config = {
    "model_name": "gpt2-xl",
}
mt = LLM(**llm_config)

Hook Config

In [ ]:
hook_config = {
    "retain_output": True,
    "retain_input": False,
    "edit_output": None,
    "clone": False,
    "detach": False,
    "device": "cpu"
}
n_layer = mt.model.config.n_layer
blocks = mt.model.transformer.h
for i in range(n_layer):
    mt.add_hook(module=blocks[i],name=f"block_{i}", **hook_config)

Dataset Config

In [14]:
from dataset.knowns import Knowns

dst = Knowns("/nvme/yangyuchen1/coding/gpt_re/data", mt.tokenizer, size=100)
dl = DataLoader(dst, batch_size=8, collate_fn=dst.collate_fn)

Loaded dataset with 1000 elements


Trainer config

In [5]:
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "auto",
    "devices" : [5],
}
trainer = pl.Trainer(**trainer_config)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.predict(mt, dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
